# Merging

## Frankenmerge / Passthrough
*concatenate layers from both models*

You can also use [MergeKit GUI Space](https://huggingface.co/spaces/arcee-ai/mergekit-gui) to merge


In [ ]:
MODEL_NAME = "MATS-frankenmerge"
yaml_config = """
slices:
  - sources:
    - model: llm-jp/llm-jp-1.3b-v1.0
      layer_range: [0, 24]
  - sources:
    - model: Qwen/Qwen1.5-0.5B
      layer_range: [2]
merge_method: passthrough
dtype: bfloat16
"""

# Save config as yaml file
with open('merge.yaml', 'w', encoding="utf-8") as f:
    f.write(yaml_config)


In [ ]:
!cd mergekit && source env/bin/activate && mergekit-yaml ../merge.yaml merge --copy-tokenizer --allow-crimes --out-shard-size 1B --lazy-unpickle

In [ ]:
from huggingface_hub import ModelCard, ModelCardData, HfApi
from jinja2 import Template
from google.colab import userdata

username = "kcoopermiller"

template_text = """
---
license: apache-2.0
tags:
- merge
- mergekit
- lazymergekit
{%- for model in models %}
- {{ model }}
{%- endfor %}
---

# {{ model_name }}

{{ model_name }} is a merge of the following models using [mergekit](https://github.com/cg123/mergekit):

{%- for model in models %}
* [{{ model }}](https://huggingface.co/{{ model }})
{%- endfor %}

## 🧩 Configuration

\```yaml
{{- yaml_config -}}
\```
"""

# Create a Jinja template object
jinja_template = Template(template_text.strip())

# Get list of models from config
data = yaml.safe_load(yaml_config)
if "models" in data:
    models = [data["models"][i]["model"] for i in range(len(data["models"])) if "parameters" in data["models"][i]]
elif "parameters" in data:
    models = [data["slices"][0]["sources"][i]["model"] for i in range(len(data["slices"][0]["sources"]))]
elif "slices" in data:
    models = [data["slices"][i]["sources"][0]["model"] for i in range(len(data["slices"]))]
else:
    raise Exception("No models or slices found in yaml config")

# Fill the template
content = jinja_template.render(
    model_name=MODEL_NAME,
    models=models,
    yaml_config=yaml_config,
    username=username,
)

# Save the model card
card = ModelCard(content)
card.save('merge/README.md')

# Defined in the secrets tab in Google Colab
api = HfApi(token=userdata.get("HF_TOKEN"))

api.create_repo(
    repo_id=f"{username}/{MODEL_NAME}",
    repo_type="model"
)
api.upload_folder(
    repo_id=f"{username}/{MODEL_NAME}",
    folder_path="merge",
)